In [4]:
import base64
import json
import pip._vendor.requests as requests
import os
import pprint as pp


def read_dependency_file(username, repository_name, file_path, github_key):
    headers = {}
    if github_key:
        headers['Authorization'] = f"token {github_key}"
        
    url = f'https://api.github.com/repos/{username}/{repository_name}/contents/{file_path}'
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    data = r.json()
    file_content = data['content']
    file_content_encoding = data.get('encoding')
    if file_content_encoding == 'base64':
        file_content = base64.b64decode(file_content).decode()

    return file_content

def get_repo_names(username, github_key):

    all_repo_names = []
    page_number = 1
    per_page = 5
    pages_remaining = True
    while_count = 0

    headers = {}
    if github_key:
        headers['Authorization'] = f"token {github_key}"

    # Make get request to fetch data of all repos and convert to JSON

    while (pages_remaining):
        
        current_url = f'https://api.github.com/users/{username}/repos?page={page_number}&per_page={per_page}'
        print('')
        print(f"while_count - {while_count}")
        r = requests.get(current_url, headers=headers)
        r.raise_for_status()
        # pp.pprint(r.headers)
        data = r.json()

        # Loop through data JSON object to extract all repo names

        for i in range(0, len(data)):   
            all_repo_names.append(data[i]['name'])
        print(all_repo_names)

        nextPattern = "rel=\"next\""
        link = r.headers.get('link')
        # print(link)

        if nextPattern in link:
            print('Found next pattern in link, continue to next page')
            page_number += 1
            while_count += 1
            print(current_url)
        else:
            print("Can't find next pattern, must be last page")
            pages_remaining = False

    pp.pprint(all_repo_names)
    return all_repo_names

def extract_repo_dependencies(all_repo_names, repos_directory, dependency_files, username, github_key):

    for i in range(0,len(all_repo_names)):

        # Create make filepath to create new repo folders:
        path = os.path.join(repos_directory, all_repo_names[i])

        # Create repo folders
        os.mkdir(path)

        # Create dependency files within repo folders 
        for j in range(0, len(dependency_files)):
            try:
                file_content = read_dependency_file(username, all_repo_names[i], dependency_files[j], github_key)
                f = open(f'{repos_directory}/{all_repo_names[i]}/{dependency_files[j]}', "w")
                f.write(file_content)
                f.close()
                print(f'{dependency_files[j]} file created.')
            except: 
                print('Could not find package.json or package-lock.json')

def main():
    GITHUB_KEY = os.environ['GITHUB_KEY']
    USERNAME = 'Harrisman05'
    DEPENDENCY_FILES = ['pom.xml','package.json','package-lock.json','requirements.txt']
    REPOS_DIRECTORY = 'repos'

    ALL_REPO_NAMES = get_repo_names(USERNAME, GITHUB_KEY)
    extract_repo_dependencies(ALL_REPO_NAMES, REPOS_DIRECTORY, DEPENDENCY_FILES, USERNAME, GITHUB_KEY)

    print(len(ALL_REPO_NAMES))

if __name__ == '__main__':
    main()



while_count - 0
['Advent-of-Code', 'Design-Patterns-Academy', 'Express.js-Academy', 'FilmFlix-Database', 'Flexbox-Academy']
Found next pattern in link, continue to next page
https://api.github.com/users/Harrisman05/repos?page=1&per_page=5

while_count - 1
['Advent-of-Code', 'Design-Patterns-Academy', 'Express.js-Academy', 'FilmFlix-Database', 'Flexbox-Academy', 'Football-Match-Data-Generator', 'Full-Stack-Library', 'Geolocation-API---Node.js', 'Grid-Academy', 'Guess-Who---Football-Quiz-App']
Found next pattern in link, continue to next page
https://api.github.com/users/Harrisman05/repos?page=2&per_page=5

while_count - 2
['Advent-of-Code', 'Design-Patterns-Academy', 'Express.js-Academy', 'FilmFlix-Database', 'Flexbox-Academy', 'Football-Match-Data-Generator', 'Full-Stack-Library', 'Geolocation-API---Node.js', 'Grid-Academy', 'Guess-Who---Football-Quiz-App', 'Harrisman05', 'JavaScript-Algorithm-Methods---Node.js', 'JavaScript-Theory', 'JS-Event-Listeners---Bubbling--Capturing-and-Stop-